In [97]:
# Group all the BL APF spectra files into subdirectories by the star they belong to. 

import pandas as pd
import shutil
import os

#apf_name_conversion = pd.read_csv('apf_name_conversion_updated.csv') # same as apf_name_conversion but has a header
apf_log_file = pd.read_csv('apf_log_full_current.csv') #pd.read_csv('./apf_log_full_16Aug2021.csv')
#apf_path = '/datax/scratch/hisaacson/data/' # '/datag/blpd0/datax/apf'
out_dir_path = './APF_spectra/all_apf_spectra/' # './APF_spectra/all_apf_spectra/'

#star_list = list(dict.fromkeys(apf_name_conversion['HIP_NAME'])) # all unique HIP names of spectra in BL APF directory
star_name_sets = list(dict.fromkeys(apf_log_file['Alt_names'])) # all unique sets of names of spectra in BL APF directory
try:
    star_name_sets.remove('Not found') # some names in the log file were not resolvable by SIMBAD and are listed as 'Not found'
except ValueError:
    print('No stars were not resolved in Simbad.')
    
def get_pathnames(name_set):
    rows = apf_log_file[apf_log_file['Alt_names'] == name_set] 
    filenames = rows["Filename"].tolist()
    paths = rows["Filepath"].tolist()
    resolvable_names = rows['Simbad_resolvable_name'].tolist() # may not be the same for all the observation of the same star (ie. if they were listed under different but both resolvable names)
    return filenames, paths, resolvable_names

not_found = []
raw_files = []
skipped = []
no_files = [] # should be empty becuase of how log file was created
for name_set in star_name_sets: 
    filenames, paths, resolvable_names = get_pathnames(name_set)
    if len(filenames) > 0 :
        print(star+': ')
        print(filenames)
        #dir_path = apf_path + '/' + star + '_spectra'
        star = resolvable_names[0].replace(' ', '_') # just pick one and remove spaces so it can become part of the filepath (so if need to query again can't neccessarily use this - use logfile)
        dir_path = out_dir_path + star + '_spectra'
        os.mkdir(dir_path)
        for i in range(len(filenames)):
            file = filenames[i]
            path = paths[i]
            filepath = path + '/' + file
            if file.startswith('r'): 
                #shutil.move('/home/azuckerman/APF_spectra/apf_spectra/'+file+'.fits', dir_path)
                try:
                    shutil.copy(filepath, dir_path)
                except FileNotFoundError:
                    print(file + '.fits not found in BL APF directory.')
                    not_found += [file]
            elif file.startswith('ucb'): 
                print('skipping raw file ' + file)
                raw_files += [file]
            else:
                print('skipping file ' + file + ' for unkown reason.')
                skipped =+ [file]
    else: 
        print('No files found for star ' + star + '.')
        no_files += [star]
        

No stars were not resolved in Simbad.
HD210610: 
['rcnc.688.fits', 'rcnc.689.fits', 'rcnc.690.fits', 'rcnc.685.fits', 'rcnc.686.fits', 'rcnc.687.fits']
HD210610: 
['rcbd.107.fits', 'rcbd.108.fits', 'rcbd.109.fits', 'rcbk.713.fits', 'rcbk.714.fits', 'rcbk.715.fits', 'rapw.195.fits']
HIP5336: 
['rcbd.094.fits', 'rcbd.095.fits', 'rcbd.096.fits', 'rcbk.709.fits', 'rcbk.710.fits', 'rcbk.711.fits', 'raqo.204.fits']
HIP6379: 
['rcbd.099.fits', 'rcbd.100.fits', 'rcbd.101.fits', 'rcbk.716.fits', 'rcbk.717.fits', 'rcbk.718.fits', 'rasn.233.fits']
HIP7078: 
['rcbg.508.fits', 'rcbg.509.fits', 'rcbg.510.fits', 'ranz.163.fits']
HIP7513: 
['rcbd.110.fits', 'rcbd.111.fits', 'rcbd.112.fits', 'rcbk.719.fits', 'rcbk.720.fits', 'rcbk.721.fits', 'rbav.266.fits', 'rbbs.228.fits', 'rbav.267.fits', 'rbbj.193.fits']
HIP7576: 
['rcbo.809.fits', 'rcbo.810.fits', 'rcbo.811.fits', 'ratl.204.fits', 'ratl.205.fits', 'rbll.241.fits']
HIP7734: 
['rcbo.806.fits', 'rcbo.807.fits', 'rcbo.808.fits', 'raqk.236.fits']
HIP79

In [21]:
import os
files = os.listdir('/datag/blpd0/datax/apf')

In [98]:
# check if empty directories
no_files = []
has_files = []
total_num_spectra = 0
for dir_name in os.listdir(out_dir_path):
    num_spectra = len(os.listdir(out_dir_path + dir_name))
    if num_spectra < 1:
        no_files += [dir_name]
    if num_spectra >= 1:
        has_files += [dir_name]
    total_num_spectra += num_spectra
        
print(str(len(no_files)) + ' directories are empty.')
print(str(len(has_files)) + ' directories are not empty.')
print(str(total_num_spectra) + ' total spectra files were placed in a subdirectory.')
print('    ' + str(len(raw_files)) + ' files skipped due to being raw.')
print('    ' + str(len(skipped)) + ' files skipped due for some other reason.')

0 directories are empty.
902 directories are not empty.
6482 total spectra files were placed in a subdirectory.
    0 files skipped due to being raw.
    0 files skipped due for some other reason.


In [ ]:
# THIS CELL IS OUTDATED! (01 Nov 2021)

apf_files = os.listdir(apf_path)
num_raw = 0
num_reduced = 0
weird_files = []
for file in apf_files:
    if file.startswith('ucb'):
        #apf_files.remove(file)
        num_raw += 1
    if file.startswith('r'):
        num_reduced += 1
    if not file.startswith('ucb') and not file.startswith('r'):
        weird_files += [file]

stars = list(apf_log_file['HIP_name'])
num_no_HIP = stars.count('None')
        
total_apf_files = len(os.listdir(apf_path))
print('In BL APF directory (' + str(total_apf_files) + ' total files): ')
print('    ' + str(num_raw) + ' raw files.')
print('    ' + str(num_reduced) + ' reduced files.')
print('    ' + str(len(weird_files)) + ' other files.')

print()
print(str(total_apf_files - total_num_spectra) + ' files were not used.')
print('    ' + str(num_raw) + ' becuase they are raw.')
print('    ' + str(num_no_HIP) + ' becuase they did not return a HIP name.')

In [92]:
weird_files

['logs', 'abcvel.ascii', 'breakthrough_apf_starlist.txt', 'apf_log.csv']